# Import Libraries
For this project, we are working with databases and queries to extract the data. Inorder to perform those functions we are using the python library called Sqlite3 which is a SQL based library. Along with sqlite3, we are also importing pandas which helps us in executing the sql queries.

In [ ]:
import sqlite3 as sql
import pandas as pd
import numpy as np

# Setting Up the Connection

In [ ]:
conn = sql.connect(r"C:\Users\shiva\Downloads\sql-murder-mystery.db")

In [ ]:
conn

# Solving the Murder
Now, In order to investigate the murder we need the crime report details.In order to retrive the crime report from the database we run the following query. We are using "WHERE" clause in order to filter out the reports and find the one which we need to investigate. The murder occurred on Jan.15, 2018, in SQL City. 

In [ ]:
query = """
SELECT *
FROM crime_scene_report
WHERE city = 'SQL City'
AND date = 20180115
AND type = 'murder'
ORDER BY Date;
"""

In [ ]:
pd.set_option('display.max_colwidth',None)

pd.read_sql_query(query,conn)

According to the report description, "Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave". We have found the two potential witness of the crime. Now we will further investigate the case by finding the details about these two witness.

# Witness Personal Details

In [ ]:
query2 = """
SELECT *
FROM person
WHERE 
    (address_street_name = 'Northwestern Dr' AND address_number = (
        SELECT MAX(address_number)
        FROM person
        WHERE address_street_name = 'Northwestern Dr'
    ))
    OR 
    (address_street_name = 'Franklin Ave' AND name LIKE '%Annabel%');

"""

In [ ]:
pd.read_sql_query(query2,conn)

During the investigation, we have find out that the one of the witness lives in the last house in 'Northwestern Dr' and the second witness lives in somewhere around 'Franklin Ave'. We have also run a background check in order to find the ssn, house number, license number.

# Interogating the Witness
Now, We have ask both the witness to come for formal interogation in order to find the potential clues regarding the murderer.

In [ ]:
query4 = """
SELECT i.transcript, p.name,i.person_id
FROM interview i
JOIN person p ON i.person_id = p.id
WHERE p.id IN (
    SELECT id
    FROM person
    WHERE 
        (address_street_name = 'Northwestern Dr' AND address_number = (
            SELECT MAX(address_number)
            FROM person
            WHERE address_street_name = 'Northwestern Dr'
        ))
        OR 
        (address_street_name = 'Franklin Ave' AND name LIKE '%Annabel%')
);

"""

In [ ]:
pd.read_sql_query(query4,conn)

Now, the Interogation is done and we have recorded the testimony of both the witness. One of the witness 'Morty Schapiro' says "I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W". Other witness 'Annabel Miller' says "I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th". Our next step in investigation leads to the gym name 'Get Fit Now' and also we have a car number as well so we have to check that as well.

# Get Fit Now Gym 
Now we have arrived at the location that our witness include in their testimonies. We have some info regarding the membership number, and the status.

In [ ]:
query5 = """
SELECT *
FROM get_fit_now_member
WHERE 
(membership_status = 'gold' AND id LIKE '%48Z%')
"""

In [ ]:
pd.read_sql_query(query5,conn)

According to our information, we have found the two suspects 'Joe Germuska' and 'Jeremy Bowers'.Both the suspects have gold membership and membership number starts with '48Z'.Now we have to check for the car number as well.

In [ ]:
query6 = """
SELECT dl.plate_number,p.name
FROM drivers_license dl
JOIN person p ON dl.id = p.license_id
WHERE dl.plate_number LIKE '%H42W%'
"""

In [ ]:
pd.read_sql_query(query6,conn)

After checking the car number, we have found the three potential suspects 'Tushar Chandra', 'Jeremy Bowers' and 'Maxine Whitely'. Out of three, 'Jeremy Bowers' is the one that we suspected the most due to the fact that he has a gold membership and the membership number that starts with '48Z'.

Now We have to investigate the fact that the second witness says that he know the murderer from his gym and he last visited the gym on janurary 9. For further investagation, we have checked for all the names that checkin in gym on 9th jan.

In [ ]:
query7 = """
SELECT gc.membership_id,gc.check_in_date,g.name
FROM get_fit_now_check_in gc
JOIN get_fit_now_member g ON gc.membership_id = g.id
WHERE gc.check_in_date LIKE '%20180109%'
"""

In [ ]:
pd.read_sql_query(query7,conn)

We have find a lot of names that visited gym on 9th janurary but the name that attracts us most is 'Jeremy Bowers' who is our potential suspect in this case and We have now enough evidence to prove that the 'Jeremy Bowers' is the murderer. We now interogate the murderer inorder to gather some information regarding the murder case.

In [ ]:
query8 = """
SELECT i.transcript, p.name
FROM interview i
JOIN person p ON i.person_id = p.id
WHERE p.name = 'Jeremy Bowers'
"""

In [ ]:
pd.read_sql_query(query8,conn)

During the interogation, we have found that jeremy bowers is hired by a women that paid him a lot of money. He says ''I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017''.

# Finding the Culprit
The case takes an unexpected turn after murderer reveals that he was hired by a women to do a murder. So we have to find that women who is the actual culprit. We have check the event checkin details and find out that there is a women who have visted the 'SQL Symphony Concert' 3 times in December 2017. We have match the identity with the data provided by the murderer like red hair, Tesla Model S car. The name of the women who have hired the murderer is "Miranda Priestly". 

In [ ]:
query9 = """
SELECT p.name,p.id
FROM facebook_event_checkin fb
JOIN person p ON fb.person_id = p.id
JOIN drivers_license dl ON p.license_id = dl.id
WHERE 
(dl.gender = 'female' AND dl.hair_color = 'red'
AND dl.car_model = 'Model S' AND fb.event_name = 'SQL Symphony Concert'
AND fb.date LIKE '%201712%')
"""

In [ ]:
pd.read_sql_query(query9,conn)

# Verdict :- Jeremy Bowers is the murderer who was hired by Miranda Priestly.